Notebook esame: 
In questo notebook mi occuperò dell' analisi degli spettri di assorbimento di due specie di molecole (CO2 e C2H2), ottenuti attraverso il metodo 'comb-locked frequency-swept synthesis' (CLFSS), il quale permette di avere simultanaemante un grande livello di precisione e di range di misura in tempi brevi.
Come primo passo importo le librerie necessarie e i dati per un plot preliminare.

In [2]:
# Import libraries
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import ipywidgets as widgets
from ipywidgets import interact_manual
from ipywidgets import interact
from scipy.signal import find_peaks
import spettro_module as sm

In [3]:
# Dizionario dei dati
dati = {}

# Costruisco i vettori
myList=['C2H2', 'CO2'] #modifica in percorsi relativi
dati['C2H2']=np.loadtxt("ATLAS_2018/C2H2_Pband_10Torr.dat",delimiter="\t", skiprows=2, usecols=range(6))
dati['CO2']=np.loadtxt("ATLAS_2018/CO2_3THz_scan_CRDS__data_2e-2Torr.dat",delimiter="\t", skiprows=2,usecols=range(6))


In [4]:
#prova plot dati
plt.close('all')

for name in myList:
    plt.figure()
    plt.plot(dati[name][:,0],dati[name][:,-1],label=name)
    plt.legend()
    plt.grid()


Ora come prima analisi è utile trovare la posizione e l'altezza dei picchi, informazioni utili da utilizzare come starting point per il fit dei modelli.

In [33]:
#plt.close('all')
#algoritmo di peaks
#per ora lasciamo da parte il calcolo della baseline
peaks={}
hmin=[.02, 6e-9]
width=10
for i,name in enumerate(myList):
    peaks[name]=[dati[name][find_peaks(dati[name][:,-1],height=hmin[i], width=width)[0],0],dati[name][find_peaks(dati[name][:,-1],height=hmin[i], width=width)[0],-1]]
    plt.figure()
    plt.plot(dati[name][:,0],dati[name][:,-1])
    plt.plot(peaks[name][0],peaks[name][1], '.')
    plt.grid()


Ora si può procedere all' analisi in due metodi, che chiameremo 'fit dei picchi' e 'fit delle bande'. Il primo consiste nell'analizzare con un fit i singoli picchi di assorbimento, mentre il secondo si procede con l'analisi dei massimi appartenenti alla stessa banda. Iniziamo col primo metodo.

In [100]:
#fit dei picchi
plt.close('all')

bline=[.005,2.5e-9]
ctrlv=[[],[]]
for i,name in enumerate(myList):
    #i=1
    #name='CO2'
    plt.figure()
    plt.grid()
    cond=(dati[name][:,-1]<=bline[i])
    dati2=dati[name][:,0]*cond
    for c,j in enumerate(peaks[name][0]):
        #perche` questa formula? voglio trovare il pt piu` vicino che sia sotto la bline (altrimenti arrivo al picco)`
        xmin=np.argmin(1/(dati2-j))
        xmax=np.argmax(1/(dati2-j)) 
        xfit=dati[name][xmin:xmax,0]
        yfit=dati[name][xmin:xmax,-1]
        if c!=len(peaks[name][0])-1:
            if np.logical_and(peaks[name][0][c+1]>xfit[-1],peaks[name][0][c-1]<xfit[0]):
                
                FWHM=xfit[yfit>=max(yfit)/2][-1]-xfit[yfit>=max(yfit)/2][0]

                #per il fit utilizzo la funzione di voigt
                p0=[np.dot(np.diff(xfit),yfit[:-1]),FWHM/(2*np.sqrt(2*np.log(2))),j,FWHM/4]
                fit,cov=curve_fit(sm.voigt,xfit,yfit,p0=p0,
                                  bounds=((p0[0],p0[1]/2,p0[2]-.1,p0[3]*.09),(p0[0]*2,p0[1],p0[2]+.1,p0[3]*2)))
                #grazie a questo vettore controllo la bonta` del guess`
                ctrlv[i].append((p0-fit)/p0)
                print((p0-fit)/p0)
                #controllo che funzioni il metodo scelto
                #plt.plot(dati[name][:,0],1/(dati2-j))
                plt.plot(xfit,yfit,'r',linewidth=1)
                plt.plot(xfit,sm.voigt(xfit,*fit),'--b',linewidth=.5)
    plt.plot(dati[name][:xmax,0],dati[name][:xmax,-1],'k',linewidth=.1)



[-4.80926671e-01  4.99854071e-01 -1.35034632e-07 -6.44720741e-01]
[-2.10962895e-01  3.34418156e-01 -1.43624320e-07  4.59740481e-04]
[-1.94871972e-01  3.19324709e-01  1.79729918e-07  5.49384313e-03]
[-1.84963561e-01  2.59668538e-01 -6.09805652e-09  1.02356865e-01]
[-4.07758971e-01  4.36945672e-01 -2.06782247e-07 -3.64050138e-01]
[-6.04644059e-02  1.44169468e-01  2.99372464e-08  4.63320745e-01]
[-3.76801518e-01  4.36664480e-01  1.94422380e-07 -4.15023947e-01]
[-3.64872215e-01  4.99824044e-01 -1.00357817e-07 -4.67167234e-01]
[-1.50957008e-01  2.38741509e-01  1.16765524e-08  1.60248269e-01]
[-1.17148427e-01  1.87307212e-01 -9.87792479e-08  3.55865015e-01]
[-3.17502226e-01  3.69653448e-01  9.94631776e-08 -2.65188602e-01]
[-1.36794825e-01  2.38700213e-01  8.33492738e-08  2.08730327e-01]
[-3.52119884e-01  4.96205592e-01 -7.66207533e-08 -5.35852365e-01]
[-4.78158686e-02  1.10004477e-01 -3.62770197e-09  5.66572956e-01]
[-2.77543479e-01  3.31403627e-01  1.68743125e-07 -1.15195376e-01]
[-3.056579

Illustriamo ora il secondo metodo, ovvero il fit delle bande attraverso la funzione di maxwell-boltzmann. La difficolta` qui sta nel distinguere le varie bande, trovando un metodo che possa farlo in autonomia

In [ ]:
#fit delle bande 
plt.close('all')
kb=1.380649 *10**-23
gmu=[196.8,190.29]
eps=[2e-1,1e-8]

vmax=[0,0]
norm=[0,0]
i=0

for name in myList:
    xb=np.copy(peaks[name][0])
    yb=np.copy(peaks[name][1])-bline[i]
    b_max=np.argmax(yb)
    vmax[i]=xb[b_max]-gmu[i]
    vmax[1]=np.sqrt(kb*(8.21917808*10**21))
    norm[i]=np.exp(1)*yb[b_max]*((abs(vmax[i])))/(((1/(np.pi))**(3/2))*(4*np.pi))
    #guess plot
    plt.figure()
    plt.grid()
    plt.plot(xb,yb, '-*')
    plt.plot(xb,sm.max_boltz(xb,vmax[i],norm[i],gmu[i])) #(xb[b_max]-gmu[i]),norm
    bcon=(abs(yb-sm.max_boltz(xb,vmax[i],norm[i],gmu[i])))*abs(xb-xb[b_max])<eps[i] #*abs(xb-xb[b_max])
    #bcon=[for j in len(xb[bcon]):  ]
    plt.plot(xb[bcon],yb[bcon],'.r')
    
    i+=1
